# Chapter 3 - Github Embeddings

In this notebook we're going to go beyond using pre-trained embeddings and models we download from the internet and start to create our own secondary models that can improve the primary model through transfer learning. We're going to train text and code embeddings based on Github's [CodeSearchNet](https://github.com/rjurney/CodeSearchNet) datasets. They include both doc strings and code for 2 million posts and while they use the data to map from text search queries to code, we'll be using it to create separate [BERT](https://arxiv.org/abs/1810.04805) embeddings to drive our Stack Overflow tagger.

The paper for CodeSearchNet is on arXiv at [CodeSearchNet Challenge: Evaluating the State of Semantic Code Search](https://arxiv.org/abs/1909.09436).

In [ ]:
import csv
import gc
from pathlib import Path
import os
import random
import sys
import warnings

from bs4 import BeautifulSoup
from nltk.tokenize.punkt import PunktSentenceTokenizer
import pandas as pd

random.seed(1337)

# Add parent directory to path
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

from lib.utils import extract_text_plain

# Disable all warnings
warnings.filterwarnings("ignore")

## Load CodeSearchNet Data

We load the entire CodeSearchNet dataset for Go, Java, PHP, Python and Ruby. While the code doesn't cover all languages I'm hoping they are diverse enough to handle other languages and so will still help performance.

In [ ]:
df = pd.DataFrame()

# Load all Gzipped JSON Lines files in the data directory
for filename in Path('../data/CodeSearchNet').glob('**/*.jsonl.gz'):
    new_df = pd.read_json(filename, lines=True)
    df = pd.concat([df, new_df])
    
    # Carefully manage memory
    del new_df
    gc.collect()

df.head()

In [ ]:
print(
    f'There are {len(df["docstring"].index):,} functions'
)

## Extract Text from Docstrings

Docstings can contain HTML, so we parse them and extract text using `BeautifulSoup`.

In [ ]:
code = df['code']
docs = df.docstring.apply(lambda x: extract_text_plain(x))

## Inspect the result of the code removal

In [ ]:
pd.set_option('max_colwidth', 500)
doc_df = pd.DataFrame({'docs': docs, 'docstring': df['docstring']})

doc_df.head(3)

## About BERT

Google BERT is described in the [BERT README](https://github.com/google-research/bert/blob/master/README.md):

> BERT is a method of pre-training language representations, meaning that we train a general-purpose "language understanding" model on a large text corpus (like Wikipedia), and then use that model for downstream NLP tasks that we care about (like question answering). BERT outperforms previous methods because it is the first unsupervised, deeply bidirectional system for pre-training NLP.

> Unsupervised means that BERT was trained using only a plain text corpus, which is important because an enormous amount of plain text data is publicly available on the web in many languages.

> Pre-trained representations can also either be context-free or contextual, and contextual representations can further be unidirectional or bidirectional. Context-free models such as word2vec or GloVe generate a single "word embedding" representation for each word in the vocabulary, so bank would have the same representation in bank deposit and river bank. Contextual models instead generate a representation of each word that is based on the other words in the sentence.

> BERT was built upon recent work in pre-training contextual representations — including Semi-supervised Sequence Learning, Generative Pre-Training, ELMo, and ULMFit — but crucially these models are all unidirectional or shallowly bidirectional. This means that each word is only contextualized using the words to its left (or right). For example, in the sentence I made a bank deposit the unidirectional representation of bank is only based on I made a but not deposit. Some previous work does combine the representations from separate left-context and right-context models, but only in a "shallow" manner. BERT represents "bank" using both its left and right context — I made a ... deposit — starting from the very bottom of a deep neural network, so it is deeply bidirectional.

## Generate CSV for BERT

The [Google BERT Github project](https://github.com/google-research/bert) is a submodule to this project, which you can checkout from within this [cloned project](https://github.com/rjurney/weakly_supervised_learning_code) with:

```bash
git submodule init
git submodule update
```

We need to generate CSV in the format that BERT expects, which is:

> Here's how to run the data generation. The input is a plain text file, with one sentence per line. (It is important that these be actual sentences for the "next sentence prediction" task). Documents are delimited by empty lines. The output is a set of tf.train.Examples serialized into TFRecord file format.

In [ ]:
sentence_tokenizer = PunktSentenceTokenizer()
sentences = docs.apply(sentence_tokenizer.tokenize)

sentences.head(2)

In [ ]:
with open('../data/sentences.csv', 'w') as f:
    
    current_idx = 0
    for idx, doc in sentences.items():
        # Insert a newline to separate documents
        if idx != current_idx:
            f.write('\n')
        # Write each sentence exactly as it appared to one line each
        for sentence in doc:
            f.write(sentence.encode('unicode-escape').decode().replace('\\\\', '\\') + '\n')

## Using `sentencepiece` to Extract a WordPiece Vocabulary

BERT needs a WordPiece vocabulary file to run, so we need to decide on a number of tokens and then run `sentencepiece` to extract a list of valid tokens.

The `sentencepiece` Pypi library isn't sufficient for our needs, we need to clone the Github repo, build and install the software to create our vocabulary.

Make sure you're in the root directory of this project and run:

```bash
git clone https://github.com/google/sentencepiece
cd sentencepiece

mkdir build
cd build
cmake ..
make -j $(nproc)
sudo make install
sudo ldconfig -v
```

Now we can use `sp_train` to create a vocabulary of our 4.7 million sentences.

In [ ]:
%%bash

cd ../models
spm_train --input="../data/sentences.csv" --model_prefix=wsl --vocab_size=20000

# Add the [CLS], [SEP], [UNK] and [MASK] tags, or pre-training will error out
echo -e "[CLS]\t0\n[SEP]\t0\n[UNK]\t0\n[MASK]\t0\n$(cat wsl.vocab)" > wsl.vocab

# Remove the numbers, just retain the tag vocabulary
cat wsl.vocab | cut -d$'\t' -f1 > wsl.stripped.vocab

## Using BERT to Pretrain a Language Model

Next we use the WordPiece vocabulary to pre-train a BERT model that we will then use, as a tranfer learning strategy, to encode the text of Stack Overflow questions.

### Creating a BERT conda environment

It is not possible to create a new conda environment from which to install `tensorflow==1.14.0`, which BERT needs, so you will need to run this code outside of this notebook, from the root directory of this project.


```bash
conda create -y -n bert python=3.7.4
conda init bash
```

Now in a new shell, change directory to the root of project:

```bash
cd /path/to/weakly_supervised_learning_code
```

Now run:

```bash
conda activate bert
pip install tensorflow-gpu==1.14.0
```

### Creating BERT Pre-Training Data

Before we can train a BERT model or extract static embedding values we need to create the pre-training data the model uses to train. The output file will be 20GB, so make sure you have the space available!

From the [BERT README](https://github.com/google-research/bert/blob/master/README.md):

> Here's how to run the data generation. The input is a plain text file, with one sentence per line. (It is important that these be actual sentences for the "next sentence prediction" task). Documents are delimited by empty lines. The output is a set of tf.train.Examples serialized into TFRecord file format.

We need to configure BERT to use our vocabulary size, so we create a `bert_config.json` file in the `bert/` directory.

```bash
# Tell BERT how many tokens to use
echo '{ "vocab_size": 20004 }' > bert/bert_config.json 
```

Then we execute the `create_pretraining_data.py` command to pre-train the network.

```bash
python bert/create_pretraining_data.py \
   --input_file=data/sentences.csv \
   --output_file=data/tf_examples.tfrecord \
   --vocab_file=models/wsl.stripped.vocab \
   --bert_config_file=bert/bert_config.json \
   --do_lower_case=False \
   --max_seq_length=128 \
   --max_predictions_per_seq=20 \
   --num_train_steps=20 \
   --num_warmup_steps=10 \
   --random_seed=1337 \
   --learning_rate=2e-5
```

Now we can run pretraining. If your GPU is only 8GB of RAM, reduce the training batch size to 16 or 24.

```bash
python bert/run_pretraining.py \
  --input_file=data/tf_examples.tfrecord \
  --output_dir=models/bert_pretraining_output \
  --do_train=True \
  --do_eval=True \
  --bert_config_file=bert/bert_config.json \
  --train_batch_size=32 \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --num_train_steps=10000 \
  --num_warmup_steps=10 \
  --learning_rate=2e-5
```

Finally, deactivate the conda environment:

```bash
conda deactivate
```